In [24]:
import sys

import requests
import bs4

from tqdm import tqdm as tq

In [2]:
catalog_url = 'https://vkuso.ru/recipes/menu/uzhin/page/%d/'

In [19]:
def get_soup_good(soup):
    goods = []

    for item in soup.find_all('div', {'class': 'views-item__line_content'}):
        href = item.h3.a.get('href')
        text = item.h3.a.text

        goods.append((text, href))
    
    return goods

def get_goods_from_page_vkusno(page):
    url = catalog_url % page
    
    try:
        r = requests.get(url)
    except requests.ConnectionError:
        return []
    
    soup = bs4.BeautifulSoup(r.content)
    
    return get_soup_good(soup)

In [32]:
def get_goods_vkusno(count=371):
    goods = []
    
    for page in tq(range(1, count+1)):
        new_goods = get_goods_from_page_vkusno(page)
        
        if new_goods:
            goods.extend(new_goods)
        else:
            print('fail!')
            break
        
    return goods
            

In [35]:
goods = get_goods_vkusno(371)

100%|██████████| 371/371 [04:25<00:00,  1.40it/s]


In [37]:
import pandas as pd

In [40]:
df = pd.DataFrame(goods)
df.columns = ['title', 'url']

In [42]:
df.to_csv('goods_vkusno.csv', sep='\t', index=False)